In [ ]:
import matplotlib as mpl
mpl.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
import sys

In [ ]:
sys.path.append('../code')
sys.path.append('../../HW3/code/') # for PCA

In [ ]:
from NeuralNet import NeuralNet, make_dir
from TransferFunctions import TanhTF
from neural_net_helpers import mnist_training, mnist_testing

In [ ]:
pca_training = pickle.load(file=open('../data/PCA_training_data.pickle', "rb"))

In [ ]:
X_train_untransfored, y_train = mnist_training()
X_test_untransfored, y_test = mnist_testing()

In [ ]:
X_train_PCA50 = np.load("../data/X_transformed_by_50_components.npy").T
X_test_PCA50 = np.load("../data/X_test_transformed_by_50_components.npy").T

In [ ]:
# columns are data points and rows are features
d, N = np.shape(X_train_PCA50)
C = np.unique(y_train).shape[0]

In [ ]:
n = NeuralNet(X=X_train_PCA50, y=y_train, 
              hidden_nodes=500, 
              hiddenTF=TanhTF, outputTF=TanhTF,
              minibatch_size=200,
              eta0 = 0.01,
              X_test = X_test_PCA50, y_test = y_test,
              PCA = pca_training,
             verbose=False)

In [ ]:
print("N: {}".format(n.N))
print("d: {}".format(n.d))
print("C: {}".format(n.C))
print("X shape: {}".format(n.X.shape))
print("y shape: {}".format(n.y.shape))
print("Y shape: {}".format(n.Y.shape))

In [ ]:
n.run(epochs=10)

In [ ]:
n.C

In [ ]:
n.results.tail()

In [ ]:
sl = n.plot_square_loss()

In [ ]:
l01 = n.plot_01_loss()

In [ ]:
#p = n.plot_weight_evolution()

In [ ]:
n.results.tail(1).T

In [ ]:
n.W1.shape

In [ ]:
n.W1[0,:].shape

In [ ]:
n.display_hidden_node_as_image(n.W1[0,:])

In [ ]:
n.display_hidden_node_as_image(n.W1[10,:])

In [ ]:
n.W1[0,:].shape

In [ ]:
def display_hidden_node_as_image(self, weights, filename=None):
    assert self.PCA is not None, "need PCA pickle loaded for use"
    assert weights.shape == (50,), "expected shape (50,); " \
                                   "got {}".format(weights.shape)
    print(weights)

    # Take it out of PCA space.
    image_vector = self.PCA.transform_number_up(weights, center=True)

    def make_image(data, path=None):
        plt.figure(figsize=(0.7,0.7))
        p=plt.imshow(data.reshape(28, 28), origin='upper', interpolation='none')
        p.set_cmap('gray_r')
        plt.axis('off')
        if path is not None:
            plt.savefig(path)
            plt.close()

    make_image(image_vector, filename)

In [ ]:
display_hidden_node_as_image(n, np.zeros(shape=(50,)))

In [ ]:
image_dir = '../figures/Q-2-1_weight_visualization'
make_dir(image_dir)

In [ ]:
import subprocess

In [ ]:
def visualize_10_W1_weights(self, folder):
    # select 10 at random
    random_indices = np.random.choice(range(self.W1.shape[0]), 
                                      size=(10,), replace=False)
    print(random_indices)
    image_num = 1
    image_paths = []
    
    for index in random_indices:
        print(index)
        x_i = self.W1[index,:]
        print(x_i[0:10])
        image_filename = 'weight_{}.pdf'.format(index)
        image_paths.append(image_filename)
        self.display_hidden_node_as_image(x_i, image_filename)
    print(image_paths)
    
    # stitch them togethe with subprocess.check_call() 
    # subprocess.call(\"\"\"mv *.png ../figures/PCA_reconstructions/\"\"\", shell=True)
    i_names_str = " ".join(image_paths)
    print(i_names_str)
    command = "convert +append " + i_names_str +' concat.png'
    print(command)
    
    # remove the individuals
    subprocess.call(command, shell=True)
    for image in image_paths:
        command = 'rm {}'.format(image)
        print(command)
        #subprocess.call(command, shell=True)
    # move the product to the correct dir
    dest_folder = os.path.join(folder, 'concat.png')
    command = 'mv concat.png {}'.format(folder)
    print(command)
    subprocess.call(command, shell=True)


In [ ]:
visualize_10_W1_weights(n, '../figures/Q-2-1_weight_visualization')

In [ ]:
n.W1.shape